In [1]:
import json
import requests

In [2]:
with open("../../secrets.json", 'r') as fr:
    content = "".join(fr.readlines())
    API_KEY = json.loads(content)['eod_histo_data_api_key']

In [5]:
params = {'api_token': API_KEY}
exchanges = requests.get(url="https://eodhistoricaldata.com/api/exchanges-list/", params=params)
len(json.loads(exchanges.text))

<Response [200]>

In [41]:
json.loads(exchanges.text)

[{'Name': 'USA Stocks',
  'Code': 'US',
  'OperatingMIC': 'XNAS, XNYS',
  'Country': 'USA',
  'Currency': 'USD'},
 {'Name': 'London Exchange',
  'Code': 'LSE',
  'OperatingMIC': 'XLON',
  'Country': 'UK',
  'Currency': 'GBP'},
 {'Name': 'Toronto Exchange',
  'Code': 'TO',
  'OperatingMIC': 'XTSE',
  'Country': 'Canada',
  'Currency': 'CAD'},
 {'Name': 'TSX Venture Exchange',
  'Code': 'V',
  'OperatingMIC': 'XTSX',
  'Country': 'Canada',
  'Currency': 'CAD'},
 {'Name': 'NEO Exchange',
  'Code': 'NEO',
  'OperatingMIC': 'NEOE',
  'Country': 'Canada',
  'Currency': 'CAD'},
 {'Name': 'Hamburg Exchange',
  'Code': 'HM',
  'OperatingMIC': 'XHAM',
  'Country': 'Germany',
  'Currency': 'EUR'},
 {'Name': 'Munich Exchange',
  'Code': 'MU',
  'OperatingMIC': 'XMUN',
  'Country': 'Germany',
  'Currency': 'EUR'},
 {'Name': 'XETRA Exchange',
  'Code': 'XETRA',
  'OperatingMIC': 'XETR',
  'Country': 'Germany',
  'Currency': 'EUR'},
 {'Name': 'Dusseldorf Exchange',
  'Code': 'DU',
  'OperatingMIC': '

In [48]:
EXCHANGE_CODE = 'LSE'
url=f"https://eodhistoricaldata.com/api/exchange-symbol-list/{EXCHANGE_CODE}?api_token={API_KEY}"
tickers = requests.get(url=url)
len(tickers.text.split("\n"))

TypeError: object of type 'Response' has no len()

In [52]:
len(tickers.text.split("\n"))

5832

In [49]:
tickers.text.split("\n")

['Code,Name,Country,Exchange,Currency,Type,Isin',
 '0A00,"Akzo Nobel N.V.",UK,LSE,EUR,"Common Stock",',
 '0A02,"Neoen S.A.",UK,LSE,EUR,"Common Stock",FR0011675362',
 '0A05,"Medacta Group S.A.",UK,LSE,CHF,"Common Stock",',
 '0A0C,"Stadler Rail AG",UK,LSE,CHF,"Common Stock",CH0002178181',
 '0A0D,"Alcon Inc.",UK,LSE,USD,"Common Stock",',
 '0A0F,"Citycon Oyj",UK,LSE,EUR,"Common Stock",FI4000369947',
 '0A0H,"Beijer Ref AB Series B",UK,LSE,SEK,"Common Stock",',
 '0A0I,"Wihlborgs Fastigheter AB (publ)",UK,LSE,SEK,"Common Stock",',
 '0A0J,"AAK AB",UK,LSE,SEK,"Common Stock",',
 '0A0K,"Nyfosa AB",UK,LSE,SEK,"Common Stock",',
 '0A1K,"NIO Inc. ADR",UK,LSE,USD,"Common Stock",US62914V1061',
 '0A1V,"Tethys Oil AB",UK,LSE,SEK,"Common Stock",SE0015949847',
 '0A28,"Prosus N.V.",UK,LSE,EUR,"Common Stock",',
 '0A29,"Solutions 30 SE",UK,LSE,EUR,"Common Stock",',
 '0A2N,"Lynas Rare Earths Ltd.",UK,LSE,AUD,"Common Stock",',
 '0A37,"Betsson AB Series B",UK,LSE,SEK,"Common Stock",SE0015672282',
 '0A39,"Karnov 

In [50]:
type_counts = {}
for elem in tickers.text.split("\n"):
    sub_elems = elem.split(",")
    if len(sub_elems) == 7:
        type_ = sub_elems[5]
        if type_ in type_counts.keys():
            type_counts[type_] += 1
        else:
            type_counts[type_] = 1
type_counts


{'Type': 1,
 '"Common Stock"': 4108,
 'FUND': 5,
 'ETF': 1688,
 'INDEX': 1,
 'ETC': 24}

In [51]:
exchange_counts = {}
for elem in tickers.text.split("\n"):
    sub_elems = elem.split(",")
    if len(sub_elems) == 7:
        type_ = sub_elems[3]
        if type_ in exchange_counts.keys():
            exchange_counts[type_] += 1
        else:
            exchange_counts[type_] = 1
exchange_counts

{'Exchange': 1, 'LSE': 5826}

In [16]:
FROM = "2002-02-23"
TO = "2022-03-23"

In [5]:
TICKER = "NVDA.US"

In [87]:
url = f"https://eodhistoricaldata.com/api/eod/{TICKER}?api_token={API_KEY}&from={FROM}&to={TO}"
quotes = requests.get(url=url)

In [71]:
quotes.text.split("\n")

['Date,Open,High,Low,Close,Adjusted_close,Volume',
 '2017-03-23,34.87,35.41,34.87,35.04,33.7932,516358',
 '2017-03-24,35.06,35.28,34.62,34.69,33.4556,587036',
 '2017-03-27,34.12,34.58,34.11,34.52,33.2917,428505',
 '2017-03-28,34.31,34.96,34.31,34.77,33.5328,439182',
 '2017-03-29,34.7,34.87,34.5,34.52,33.2917,575910',
 '2017-03-30,34.5,35,34.33,34.84,33.6003,902289',
 '2017-03-31,34.65,35.08,34.59,34.72,33.4846,567693',
 '2017-04-03,33.69,34.31,33.33,34.07,32.8577,1917200',
 '2017-04-04,34.04,34.78,33.97,34.64,33.4074,730163',
 '2017-04-05,34.83,35.5089,34.58,34.63,33.3978,1666198',
 '2017-04-06,34.87,34.88,34.23,34.47,33.2435,992091',
 '2017-04-07,34.41,34.99,34.13,34.7,33.4653,514175',
 '2017-04-10,34.85,34.92,34.42,34.56,33.3303,364289',
 '2017-04-11,34.46,34.54,33.83,34.33,33.1085,1031006',
 '2017-04-12,34.2,34.31,33.85,34.05,32.8384,588135',
 '2017-04-13,33.99,34.2,33.75,33.98,32.7709,793763',
 '2017-04-17,34.01,34.58,34.01,34.5,33.2724,522525',
 '2017-04-18,34.43,34.89,34.14,34.79

In [88]:
url = f"https://eodhistoricaldata.com/api/div/{TICKER}?api_token={API_KEY}&from={FROM}&fmt=json"
dividends = requests.get(url=url)

In [89]:
dividends.json()

[{'date': '2017-05-19',
  'declarationDate': None,
  'recordDate': '2017-05-23',
  'paymentDate': '2017-06-14',
  'period': 'Quarterly',
  'value': 0.035,
  'unadjustedValue': 0.14,
  'currency': 'USD'},
 {'date': '2017-08-22',
  'declarationDate': '2017-08-10',
  'recordDate': '2017-08-24',
  'paymentDate': '2017-09-18',
  'period': 'Quarterly',
  'value': 0.035,
  'unadjustedValue': 0.14,
  'currency': 'USD'},
 {'date': '2017-11-22',
  'declarationDate': '2017-11-09',
  'recordDate': '2017-11-24',
  'paymentDate': '2017-12-15',
  'period': 'Quarterly',
  'value': 0.0375,
  'unadjustedValue': 0.15,
  'currency': 'USD'},
 {'date': '2018-02-22',
  'declarationDate': '2018-02-08',
  'recordDate': '2018-02-23',
  'paymentDate': '2018-03-16',
  'period': 'Quarterly',
  'value': 0.0375,
  'unadjustedValue': 0.15,
  'currency': 'USD'},
 {'date': '2018-05-23',
  'declarationDate': '2018-05-10',
  'recordDate': '2018-05-24',
  'paymentDate': '2018-06-15',
  'period': 'Quarterly',
  'value': 0.

In [90]:
url = f"https://eodhistoricaldata.com/api/splits/{TICKER}?api_token={API_KEY}&from={FROM}"
splits = requests.get(url=url)

In [91]:
splits.text

'Date,"Stock Splits"\n2021-07-20,4.000000/1.000000\n49'

In [24]:
for API_KEY in ["NVDA.US", "TSLA.US", "MSFT.US", "AMZN.US", "NFLX.US"]:
    url = f"https://eodhistoricaldata.com/api/eod/{TICKER}?api_token={API_KEY}&from={FROM}&to={TO}"
    quotes = requests.get(url=url)
    # print("-", end="")
    # url = f"https://eodhistoricaldata.com/api/div/{TICKER}?api_token={API_KEY}&from={FROM}&fmt=json"
    # dividends = requests.get(url=url)
    # print("-", end="")
    # url = f"https://eodhistoricaldata.com/api/splits/{TICKER}?api_token={API_KEY}&from={FROM}"
    # splits = requests.get(url=url)
    print("-")


-
-
-
-
-
